<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=a5fb19770232c2e2094bc62407abc52e07f9e555e39b5efe01effa212426d6eb
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-18 10:04:25
-------------------
qualified stocks: 87
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.44 L
Current:  1.43 C
-------------------
Today PnL: 54.05 K (0.38%)
Current PnL: -16.42 L (-11.04%)
CY Booked + Current PnL: -5.14 L (-3.46%)
-------------------
Total profit:  2.20 L
Total loss:  -18.62 L
-------------------
Total Booked + Current PnL: 21.84 L (17.95%)
Total Booked PnL: 38.26 L (31.44%)
Curr Year Booked PnL: 11.28 L (7.89%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 82.04 L (57.36%)
Deployed:  1.22 C
Current:  1.43 C
CAGR/XIRR %: 10.01%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,APOLLOHOSP,8285.00,-13.64,56.0,H-LC,10.85,172557.0,16380.0,9715.0,-0.56,10.49,5.63,16.71,22.0,1.69,1.25,29.75,X40N,BTT,HEALTHCARE
77,TTKPRESTIG,770.00,105.76,61.0,M-SC,3.96,88654.0,-12123.0,12217.0,-0.00,-12.03,13.78,0.09,245.0,-0.99,0.64,16.47,OX40N,NTT,DURABLES
57,RELIANCE,1533.00,-11.41,61.0,H-LC,5.19,221473.0,11107.0,17674.0,0.42,5.28,7.98,13.68,37.0,0.63,1.60,22.68,XY25,NTT,REFINERIES
50,NATIONALUM,247.44,-43.66,62.0,H-MC,6.79,112318.0,11814.0,18824.0,-0.63,11.75,16.76,30.49,79.0,0.63,0.81,47.97,MH,ATH,METALS
42,ITC,452.00,-37.50,52.0,H-LC,2.52,201492.0,1354.0,19545.0,0.67,0.68,9.70,10.44,4.0,0.07,1.45,6.28,X40,NTT,FMCG


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,IEX,219.00,-29.69,56.0,H-SC,17.07,201162.0,1208.0,96015.0,-0.13,0.60,47.73,48.63,115.0,0.01,1.45,12.23,XR,NTT,MISC
35,ICICIPRULI,790.00,-20.88,44.0,H-MC,2.24,136142.0,354.0,42395.0,0.38,0.26,31.14,31.48,107.0,0.01,0.98,12.41,X40,ATH,INSURANCE
30,HAVELLS,2069.16,0.06,57.0,H-MC,1.87,247884.0,153.0,74911.0,-0.28,0.06,30.22,30.30,92.0,0.00,1.79,13.36,X40,ATH,ELECTRICAL
52,PGHH,17973.08,-30.55,54.0,H-MC,4.20,200910.0,90.0,68691.0,0.07,0.04,34.19,34.25,80.0,0.00,1.45,5.24,X40,ATH,FMCG


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-17.18,39.0,H-LC,6.39,298233.0,-17609.0,130268.0,-0.76,-5.58,43.68,35.67,5.0,-0.14,2.15,7.51,X40N,ATH,FMCG
6,ASIANPAINT,3465.66,-12.93,48.0,H-LC,7.84,219833.0,-31935.0,85141.0,0.18,-12.68,38.73,21.13,27.0,-0.38,1.59,18.25,X40,ATH,PAINTS
42,ITC,452.00,-37.50,52.0,H-LC,2.52,201492.0,1354.0,19545.0,0.67,0.68,9.70,10.44,4.0,0.07,1.45,6.28,X40,NTT,FMCG
20,CIPLA,1795.00,-19.78,52.0,H-LC,5.76,214200.0,9700.0,31723.0,0.25,4.74,14.81,20.25,10.0,0.31,1.55,13.89,X40N,BTT,PHARMA
80,UNITDSPR,1644.00,-12.94,52.0,H-LC,7.80,228210.0,-5934.0,54565.0,-0.77,-2.53,23.91,20.77,86.0,-0.11,1.65,4.21,X40N,NTT,BREWERIES


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,0.87,71.0,H-LC,1.29,250020.0,3869.0,75331.0,2.45,1.57,30.13,32.18,16.0,0.05,1.80,38.18,X200,ATH,IT
42,ITC,452.0,-37.50,52.0,H-LC,2.52,201492.0,1354.0,19545.0,0.67,0.68,9.70,10.44,4.0,0.07,1.45,6.28,X40,NTT,FMCG
51,NESTLEIND,1377.0,-8.60,56.0,H-LC,4.38,281174.0,15748.0,41051.0,-0.22,5.93,14.60,21.40,11.0,0.38,2.03,14.18,XY25,NTT,FMCG
57,RELIANCE,1533.0,-11.41,61.0,H-LC,5.19,221473.0,11107.0,17674.0,0.42,5.28,7.98,13.68,37.0,0.63,1.60,22.68,XY25,NTT,REFINERIES
20,CIPLA,1795.0,-19.78,52.0,H-LC,5.76,214200.0,9700.0,31723.0,0.25,4.74,14.81,20.25,10.0,0.31,1.55,13.89,X40N,BTT,PHARMA


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,ATULAUTO,844.00,4002.70,67.0,M-SC,5.76,127888.0,-19480.0,71298.0,3.48,-13.22,55.75,35.16,236.0,-0.27,0.92,30.63,XY24,NTT,AUTO
47,LAOPALA,464.00,118.16,55.0,H-SC,2.34,99893.0,-31212.0,82462.0,2.65,-23.81,82.55,39.09,142.0,-0.38,0.72,28.65,AR,NTT,CERAMICS
11,BANDHANBNK,400.00,-9.90,54.0,H-SC,5.14,225736.0,-52823.0,310274.0,2.54,-18.96,137.45,92.42,151.0,-0.17,1.63,30.46,XY24,NTT,BANKS
48,LTIM,7230.20,0.87,71.0,H-LC,1.29,250020.0,3869.0,75331.0,2.45,1.57,30.13,32.18,16.0,0.05,1.80,38.18,X200,ATH,IT
16,BSOFT,836.99,-11.70,58.0,H-SC,15.01,99068.0,-41585.0,114364.0,1.97,-29.57,115.44,51.74,131.0,-0.36,0.71,12.97,XR,ATH,IT


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,HINDZINC,730.22,20.84,59.0,M-LC,1.35,197403.0,-7673.0,120238.0,-1.23,-3.74,60.91,54.89,52.0,-0.06,1.42,19.45,X5K,ATH,METALS
27,GREENPANEL,537.00,235.92,64.0,M-SC,2.40,149914.0,-33164.0,112136.0,-0.89,-18.11,74.80,43.14,230.0,-0.30,1.08,41.01,XY24,NTT,MISC
55,RAJOOENG,143.33,-35.87,39.0,H-SC,11.57,97130.0,-5370.0,46205.0,-0.89,-5.24,47.57,39.83,114.0,-0.12,0.70,1.53,AR,ATH,MISC
3,ALKYLAMINE,4546.37,-4.17,48.0,H-SC,12.75,89259.0,-11704.0,106236.0,-0.86,-11.59,119.02,93.63,148.0,-0.11,0.64,32.95,SR,ATH,CHEMICALS
80,UNITDSPR,1644.00,-12.94,52.0,H-LC,7.80,228210.0,-5934.0,54565.0,-0.77,-2.53,23.91,20.77,86.0,-0.11,1.65,4.21,X40N,NTT,BREWERIES


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.0,105.76,61.0,M-SC,3.96,88654.0,-12123.0,12217.0,-0.00,-12.03,13.78,0.09,245.0,-0.99,0.64,16.47,OX40N,NTT,DURABLES
78,UJJIVANSFB,60.0,96.03,56.0,H-SC,9.54,124200.0,-18279.0,37794.0,0.55,-12.83,30.43,13.70,163.0,-0.48,0.90,46.59,OX40N,NTT,BANKS
19,CERA,9475.0,-19.09,53.0,H-SC,2.07,147580.0,-28323.0,70351.0,0.67,-16.10,47.67,23.89,149.0,-0.40,1.06,27.45,OX40N,NTT,CERAMICS
45,KANSAINER,340.0,-67.42,61.0,H-SC,1.44,225162.0,-44505.0,80833.0,0.28,-16.50,35.90,13.47,138.0,-0.55,1.62,13.88,XY24,NTT,PAINTS
72,TATAELXSI,9161.0,-19.17,56.0,H-MC,7.63,109326.0,-22509.0,64732.0,1.25,-17.07,59.21,32.03,98.0,-0.35,0.79,22.45,OX40N,NTT,IT


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-24.83,61.0,H-LC,12.31,299888.0,-46068.0,120915.0,0.55,-13.32,40.32,21.64,1.0,-0.38,2.16,6.24,X40,ATH,IT
40,INFY,2275.00,-16.73,55.0,H-LC,8.66,324271.0,11383.0,160320.0,0.73,3.64,49.44,54.87,3.0,0.07,2.34,10.53,X40,BTT,IT
42,ITC,452.00,-37.50,52.0,H-LC,2.52,201492.0,1354.0,19545.0,0.67,0.68,9.70,10.44,4.0,0.07,1.45,6.28,X40,NTT,FMCG
83,VBL,671.64,-17.18,39.0,H-LC,6.39,298233.0,-17609.0,130268.0,-0.76,-5.58,43.68,35.67,5.0,-0.14,2.15,7.51,X40N,ATH,FMCG
1,ABB,7934.00,-37.24,70.0,H-LC,10.31,262349.0,730.0,118477.0,1.48,0.28,45.16,45.57,7.0,0.01,1.89,11.63,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,44.35,63.0,M-MC,6.96,227614.0,2652.0,164997.0,-0.43,1.18,72.49,74.53,192.0,0.02,1.64,33.81,XY24,BTT,STEEL
15,BLUESTARCO,2326.38,14.07,66.0,H-MC,3.58,196950.0,32280.0,35687.0,1.51,19.60,18.12,41.28,89.0,0.90,1.42,29.15,X40N,ATH,AC
84,VOLTAS,1918.49,0.60,56.0,H-MC,2.47,210990.0,19248.0,76779.0,-0.38,10.04,36.39,50.08,99.0,0.25,1.52,17.52,XY25,ATH,AC
18,CAMS,5211.76,0.38,66.0,H-SC,3.81,114044.0,12040.0,31887.0,1.48,11.80,27.96,43.06,122.0,0.38,0.82,30.94,X40N,ATH,MISC
30,HAVELLS,2069.16,0.06,57.0,H-MC,1.87,247884.0,153.0,74911.0,-0.28,0.06,30.22,30.30,92.0,0.00,1.79,13.36,X40,ATH,ELECTRICAL


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,247.44,-43.66,62.0,H-MC,6.79,112318.0,11814.0,18824.0,-0.63,11.75,16.76,30.49,79.0,0.63,0.81,47.97,MH,ATH,METALS
85,WHIRLPOOL,2270.00,-40.29,53.0,M-SC,0.49,100928.0,9430.0,69327.0,0.31,10.31,68.69,86.07,223.0,0.14,0.73,46.56,XR,NTT,DURABLES
37,INDIAMART,4850.92,-51.84,42.0,H-SC,11.68,134419.0,11083.0,122684.0,-0.57,8.99,91.27,108.45,119.0,0.09,0.97,33.35,AR,ATH,MISC
86,WIPRO,420.00,-8.80,62.0,M-LC,5.62,160295.0,9350.0,100104.0,1.73,6.19,62.45,72.51,53.0,0.09,1.16,12.59,XR,NTT,IT
12,BANKINDIA,190.00,-29.19,66.0,H-MC,5.97,184006.0,4198.0,108214.0,1.80,2.33,58.81,62.52,88.0,0.04,1.33,37.22,XR,NTT,BANKS


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RAJESHEXPO,518.00,1898.74,68.0,L-SC,3.04,54537.0,-82640.0,82733.0,0.47,-60.24,151.70,0.07,267.0,-1.00,0.39,35.64,OX40N,NTT,JEWELLERY
7,ASIANTILES,137.00,6716.67,53.0,L-SC,5.18,74764.0,-19046.0,95661.0,0.91,-20.30,127.95,81.67,269.0,-0.20,0.54,44.78,XR,NTT,CERAMICS
22,COFFEEDAY,80.00,-43.66,51.0,L-SC,42.31,84826.0,-28723.0,68777.0,-0.34,-25.30,81.08,35.27,268.0,-0.42,0.61,107.61,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-4.17,48.0,H-SC,12.75,89259.0,-11704.0,106236.0,-0.86,-11.59,119.02,93.63,148.0,-0.11,0.64,32.95,SR,ATH,CHEMICALS
49,MASFIN,398.61,-16.89,52.0,H-SC,12.99,94515.0,-3465.0,25065.0,-0.41,-3.54,26.52,22.05,152.0,-0.14,0.68,37.38,XR,ATH,FINANCE


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,ATULAUTO,844.0,4002.70,67.0,M-SC,5.76,127888.0,-19480.0,71298.0,3.48,-13.22,55.75,35.16,236.0,-0.27,0.92,30.63,XY24,NTT,AUTO
81,VAIBHAVGBL,521.0,64.74,71.0,H-SC,7.98,142794.0,-39981.0,151047.0,1.45,-21.87,105.78,60.77,125.0,-0.26,1.03,27.89,XR,NTT,JEWELLERY
22,COFFEEDAY,80.0,-43.66,51.0,L-SC,42.31,84826.0,-28723.0,68777.0,-0.34,-25.30,81.08,35.27,268.0,-0.42,0.61,107.61,XR,NTT,HOTELS
61,SAMMAANCAP,326.0,-171.59,62.0,M-SC,2.62,82236.0,-19974.0,113362.0,1.47,-19.54,137.85,91.37,208.0,-0.18,0.59,33.50,XY25,NTT,FINANCE
75,TITAGARH,1548.0,1.40,69.0,H-SC,3.26,166268.0,-25747.0,104632.0,0.84,-13.41,62.93,41.08,158.0,-0.25,1.20,40.89,XY24,NTT,ENGINEERING


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.81
1,25,43.29
2,50,73.71


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.98
LC    33.00
MC    22.93
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.94
X40      15.21
X40N     13.01
XR       11.62
AR        8.68
OX40N     8.14
XY25      8.12
X200      1.80
MH        1.69
X5K       1.42
SR        1.28
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    28.95
H-LC    26.34
H-MC    20.01
M-SC    13.49
M-LC     5.60
M-MC     2.62
L-SC     1.54
L-LC     1.06
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.52,-12.11,73.24
FMCG,11.70,-4.36,40.87
FINANCE,9.62,-15.34,60.51
MISC,7.40,-11.96,74.23
BANKS,6.27,-14.86,75.29
PAINTS,5.79,-14.98,32.11
ELECTRICAL,5.70,-4.22,61.65
AC,3.65,4.47,27.88
INSURANCE,3.38,-7.08,43.81


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2831235.0,22
XR,1298958.0,14
AR,1292806.0,9
X40,739436.0,10
X40N,609055.0,10
OX40N,517429.0,10
XY25,395890.0,6
SR,243178.0,2
X5K,120238.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3393996.0,29
M-SC,1374001.0,17
H-LC,1225730.0,15
H-MC,1178498.0,15
M-LC,373433.0,4
M-MC,314313.0,2
L-SC,247171.0,3
L-MC,60462.0,1
L-LC,36707.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1020290.0      6
M-SC       XY24       794375.0      7
H-SC       AR         782370.0      5
           XR         752079.0      7
H-MC       XY24       488972.0      4
H-LC       X40        455624.0      5
           AR         349585.0      2
M-MC       XY24       314313.0      2
H-SC       X40N       286552.0      4
           OX40N      247596.0      4
           SR         243178.0      2
H-LC       X40N       226271.0      4
H-MC       X40        214428.0      4
           XY25       187096.0      2
L-SC       XR         164438.0      2
M-SC       AR         160851.0      2
M-LC       XY24       153091.0      2
M-SC       OX40N      122368.0      4
M-LC       X5K        120238.0      1
M-SC       XR         113661.0      2
           XY25       113362.0      1
H-MC       XR         108214.0      1
M-LC       XR         100104.0      1
H-MC       X40N        96232.0      2
L-SC       OX40N       82733.0      1
H-LC       X200        75331.0      1
M-SC       X40         69384.0      1
H-MC       OX40N       64732.0      1
H-SC       MH          61931.0      1
L-MC       XR          60462.0      1
H-LC       XY24        60194.0      1
           XY25        58725.0      2
L-LC       XY25        36707.0      1
H-MC       MH          18824.0      1

In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 53.0 seconds
